In [56]:
import mosfit
import numpy as np
import time

import astropy.cosmology as cosmo
import astropy.units as u

from gwtoolbox import tools_earth
from scipy.stats import truncnorm

**Utility Methods**

In [24]:
def jansky(mab):
    return 10**((8.9-mab)/2.5)

**Class Methods and Features**

*Gravitational Wave Generator*

In [64]:
def gwtoolbox_gen(H0_true):
    Om0 = 0.31
    T0 = 2.725
    # generate desired cosmology
    cosmos = tools_earth.set_cosmology(None, H0_true, Om0, T0)
    # R0, tau, m_mean, m_sclae, m_low, m_high, chi_sigma
    # R0: merger rate [/yr/Gpc3] 
    # tau: Delay time from formation to merger [Gyr]
    # mass mean, standard deviation, low, high [solar masses]
    # dispersion of effective spin
    # be sure to keep these consistent with MOSFIT!
    # find sauce to motivate the selected values
    BNS_par = [300,3,1.4,0.5,1.1,2.5,0.1]
    # summon the desired tool (LIGO)
    Tools = tools_earth.Tools(detector_type='ligo', event_type='nsns',\
                              population=BNS_par, cosmos=cosmos)
    # set generator parameters
    time_obs = 60*24*365 # units of minutes of observation
    rho_cri = 8 # signal-to-noise ratio cutoff
    tot_num = Tools.total_number(time_obs, rho_cri)
    # returns keys 'z','D','m1','m2','χ','dz','dm1','dm2','dχ','dD','dtb'
    list_det = Tools.list_with_errors_df(time_obs, rho_cri, dtp=True)
    params = [list_det['z'], list_det['D'], list_det['m1'], list_det['m2'],\
              list_det['χ'], list_det['dz'], list_det['dm1'], list_det['dm2'],\
              list_det['dχ'], list_det['dD'], list_det['dtb']]
    return params

I will want to turn this into a one-off O4 generator as follows: 
1. Generate all 20-something gravitational waves for one year
2. determine the fraction which have GW detections
3. use mosfit to determine the fraction which have EM detections

For the time being, let us take a single event and run our forward model for this event

In [65]:
o4_all = gwtoolbox_gen(70)

In [73]:
# z, DL, m1, m2, etc...
single_event = np.array(o4_all).T[0]

At this stage, I sample all of my favorite parameters from their priors. In the baby model, I will only sample $H_0$ and $v$, while leaving all other parameters fixed.

In [72]:
single_event

array([ 6.26481126e-02,  2.80912383e+02,  2.14994146e+00,  1.39287258e+00,
       -7.98370266e-02,  3.13240563e-02,  1.07497073e-01,  6.96436289e-02,
        2.32625806e-04,  1.46613391e+02,  1.55000000e-01])

In [52]:
# I am still unsure as to the effect of the limiting magnitude here, besides setting a cutoff

my_fitter = mosfit.fitter.Fitter(quiet=False, test=True, offline=False)
# my_fitter = mosfit.fitter.Fitter(limiting_magnitude=23.3, quiet=False, test=True, offline=False)

In [53]:
fixed_params = {"ebv": 2.2, "rvhost": 3.1, "frad": 0.999, "nnhost": 1e18,\
              "texplosion": -0.01, "temperature": 2500, "kappa_red": 10,\
              "kappa_blue": 0.5, "kappagamma": 10000.0, "Mchirp": 1.188,\
              "q": 0.92, "cos_theta": 0.5, "cos_theta_open": 0.707107,\
              "disk_frac": 0.15, "radius_ns": 11.0, "alpha": 1.0,\
              "Mtov": 2.2, "cos_theta_cocoon": 0.5, "tshock": 1.7,\
              "temperature_shock": 100, "lumdist": 400, "redshift": 0.9}
              
# ebv: host galaxy extinction, sampled from RA/DEC
# rvhost: host galaxy extinction (?)
# frad: no idea
# nnhost: no idea
# texplosion: when the explosion occurs (?)
# temperature: surface temperature of NS (?)
# kappa_red: no idea
# kappa_blue: no idea
# kappagamma: no idea
# Mchirp: the chirp mass
# q: M1/M2 ratio
# cos_theta: orientation angle
# cos_theta_open: no idea
# disk_frac: no idea
# radius_ns: radius of NS (?)
# alpha: no idea
# Mtov: no idea
# cos_theta_cocoon: no idea
# tshock: time of shock (?)
# temperature_shock: temperature of shock (?)

In [54]:
def light_curve(fitfunc, fixed_params):
    """
    Given a fit function and a list of fixed parameters, generates a light curve using mosfit 
    and returns whether a detection is made.
    args:
    - fixed_params: dictionary of fixed parameters
    returns:
    - detection: bool
    
    To do: this should also take the observing time window in units of days after the explosion,
    the magnitude cutoff of each observing band, and probably something else.
    """
    # convert dictionary to list
    # consider turning this into a utility function
    fpar = ["ebv", fixed_params['ebv'], "rvhost", fixed_params['rvhost'],\
            "frad", fixed_params['frad'], "nnhost", fixed_params['nnhost'],\
            "texplosion", fixed_params['texplosion'],\
            "temperature", fixed_params['temperature'],\
            "kappa_red", fixed_params['kappa_red'],\
            "kappa_blue", fixed_params['kappa_blue'],\
            "kappagamma", fixed_params['kappagamma'],\
            "Mchirp", fixed_params['Mchirp'],\
            "q", fixed_params['q'], "cos_theta", fixed_params['cos_theta'],\
            "cos_theta_open", fixed_params['cos_theta_open'],\
            "disk_frac", fixed_params['disk_frac'],\
            "radius_ns", fixed_params['radius_ns'],\
            "alpha", fixed_params['alpha'], "Mtov", fixed_params['Mtov'],\
            "cos_theta_cocoon", fixed_params['cos_theta_cocoon'],\
            "tshock", fixed_params['tshock'], "temperature_shock",\
            fixed_params['temperature_shock'],\
            "lumdist", fixed_params['lumdist'], "redshift", fixed_params['redshift']]
    # create kwargs dict using fixed_params
    kwargs = dict(events=[], models=['bns_generative'],\
              max_time=4, band_list="z", band_systems="AB", iterations=0, num_walkers=1,\
              smooth_times=4, suffix="jupyter_test", user_fixed_parameters=fpar,\
             quiet=True)
    # run mosfit
    entries, ps, lnprobs = my_fitter.fit_events(**kwargs)
    # get observed magnitudes
    # see if there is any way to do this without a for loop
    obs_mags = []
    for entry in entries[0][0]['photometry']:
        if entry['system'] == 'AB':
            obs_mags += [float(entry['magnitude'])]
    print(obs_mags)
    # magnitude probability of detection
    sig10 = 21.3
    mclip_a = 0
    mclip_b = 100
    m_std = abs((jansky(sig10)) / 10)
    a, b = mclip_a / m_std, mclip_b / m_std
    v = truncnorm(a,b)
    # see if the lowest magnitude beats our probabilistic detection threshold
    flux_obs = jansky(np.array(obs_mags))
    obs_probs = v.cdf(flux_obs/m_std)
    mprob = np.random.uniform(0,1)
    detect = mprob < obs_probs
    return True in detect

In [ ]:
def simulator(v, H0):
    # use results from GWToolbox to get the true DL, M1, M2
    TDL = single_event[1]
    M1 = single_event[2]
    M2 = single_event[3]
    # derived mass quantities
    Q = M1/M2
    Mchirp = ((M1*M2)**3/(M1+M2))**(1/5)
    # create a cosmology from H0 and generate the true redshift
    # in this sample cosmology
    # assume omega_m=0.31 (make sure this is consistent with GWTB)
    universe = cosmo.FlatLambdaCDM(H0, 0.31)
    # "z in cosmology"
    ZIC = z_at_value(universe.luminosity_distance, TDL*1e6)
    fixed_params = {"ebv": 2.2, "rvhost": 3.1, "frad": 0.999, "nnhost": 1e18,\
              "texplosion": -0.01, "temperature": 2500, "kappa_red": 10,\
              "kappa_blue": 0.5, "kappagamma": 10000.0, "Mchirp": Mchirp,\
              "q": Q, "cos_theta": v, "cos_theta_open": 0.707107,\
              "disk_frac": 0.15, "radius_ns": 11.0, "alpha": 1.0,\
              "Mtov": 2.2, "cos_theta_cocoon": 0.5, "tshock": 1.7,\
              "temperature_shock": 100, "lumdist": TDL, "redshift": ZIC}
    det = light_curve(my_fitter, fixed_params)
    return {}

In [55]:
light_curve(my_fitter, fixed_params)


Basic model file:
  C:\Users\sgagn\Anaconda3\envs\kn\Lib\site-packages\mosfit\models\model.json
Model file:
  C:\Users\sgagn\Anaconda3\envs\kn\Lib\site-packages\mosfit\models\bns_generative\bns_generative.json
Parameter file:
  models\bns_generative\parameters.json
No event specified, generating dummy data.

Loading data...

Loading module `transient` [ 0% ]
Constructing observation array [ 0.0% ]
Constructing observation array [ 25.0% ]
Constructing observation array [ 50.0% ]
Constructing observation array [ 75.0% ]

Loading module `lumdist` [ 2% ]

Loading module `redshift` [ 3% ]

Loading module `alltimes` [ 5% ]

Loading module `texplosion` [ 7% ]

Loading module `resttimes` [ 8% ]

Loading module `Mchirp` [ 10% ]

Loading module `q` [ 12% ]

Loading module `disk_frac` [ 13% ]

Loading module `radius_ns` [ 15% ]

Loading module `alpha` [ 17% ]

Loading module `Mtov` [ 18% ]

Loading module `errMdyn` [ 20% ]

Loading module `errMdisk` [ 22% ]

Loading module `densetimes` [ 23% ]



False

In [50]:
en[0][0]['models']

[OrderedDict([('code', 'MOSFiT'),
              ('date', '2021/12/01'),
              ('name', 'bns_generative'),
              ('realizations',
               [OrderedDict([('alias', '1'),
                             ('parameters',
                              OrderedDict([('M1',
                                            {'value': 1.4229970179701683}),
                                           ('M2',
                                            {'value': 1.3091572565325549}),
                                           ('Mchirp',
                                            OrderedDict([('latex',
                                                          'M_{\\rm chirp}\\,(M_\\odot)'),
                                                         ('log', False),
                                                         ('value', 1.188)])),
                                           ('Mtov',
                                            OrderedDict([('latex',
                                 